## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [66]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore', FutureWarning)

We will be using the required dataset for this assignment.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. <br/> 
Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's load the CSV data and read it into a <em>pandas</em> dataframe.


In [67]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [68]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [69]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [70]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [71]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [72]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [73]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [74]:
number_of_columns = predictors.shape[1] # number of predictors
number_of_columns

8

## Import Keras


#### Let's go ahead and import the Keras library


In [75]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [76]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [77]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(number_of_columns,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.


Let's import scikit-learn in order to randomly split the data into a training and test sets


<a id='item34'></a>


In [78]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing


In [79]:
x_training, x_testing, y_training, y_testing = train_test_split(predictors, target, test_size=0.3, random_state=50)

## Train and Test the Network


Let's call the function now to create our model.


In [80]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [81]:
# fit the model
epochs = 50
model.fit(x_training, y_training, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 376us/step - loss: 31326.5833
Epoch 2/50
721/721 [==============================] - 0s 138us/step - loss: 10525.1090
Epoch 3/50
721/721 [==============================] - 0s 107us/step - loss: 2729.0634
Epoch 4/50
721/721 [==============================] - 0s 105us/step - loss: 1682.5331
Epoch 5/50
721/721 [==============================] - 0s 89us/step - loss: 1309.4328
Epoch 6/50
721/721 [==============================] - 0s 84us/step - loss: 1030.5464
Epoch 7/50
721/721 [==============================] - 0s 75us/step - loss: 866.4514
Epoch 8/50
721/721 [==============================] - 0s 64us/step - loss: 751.5871
Epoch 9/50
721/721 [==============================] - 0s 76us/step - loss: 660.8149
Epoch 10/50
721/721 [==============================] - 0s 63us/step - loss: 591.6512
Epoch 11/50
721/721 [==============================] - 0s 68us/step - loss: 529.4862
Epoch 12/50
721/721 [==============================] - 0s 60us

Next we need to evaluate the model on the test data.


In [82]:
loss_val = model.evaluate(x_testing, y_testing)
y_prediction = model.predict(x_testing)
loss_val

309/309 [==============================] - 0s 305us/step


81.82181291055525

In [ ]:
Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.


In [83]:
from sklearn.metrics import mean_squared_error

In [84]:
mean_square_error = mean_squared_error(y_testing, y_prediction)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

81.82180922537891 0.0


In [85]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    x_training, x_testing, y_training, y_testing = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(x_training, y_training, epochs=epochs, verbose=0)
    MeanSquaredError = model.evaluate(x_testing, y_testing, verbose=0)
    print("Mean Squared Error "+str(i+1)+": "+str(MeanSquaredError))
    y_prediction = model.predict(x_testing)
    mean_square_error = mean_squared_error(y_testing, y_prediction)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("This is the mean with a standard deviation of " +str(total_mean_squared_errors) + " mean squared errors. Total number of training epochs: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Squared Error 1: 67.72841251706615
Mean Squared Error 2: 86.00057810564257
Mean Squared Error 3: 58.78793821365702
Mean Squared Error 4: 65.83779336873768
Mean Squared Error 5: 56.38292254599167
Mean Squared Error 6: 51.84291949472767
Mean Squared Error 7: 57.77233225010745
Mean Squared Error 8: 41.41064919777287
Mean Squared Error 9: 44.98823722666521
Mean Squared Error 10: 47.712244496762175
Mean Squared Error 11: 44.467673033186536
Mean Squared Error 12: 47.88969080656477
Mean Squared Error 13: 53.609192771047454
Mean Squared Error 14: 49.03638078016756
Mean Squared Error 15: 46.92531302986021
Mean Squared Error 16: 40.42894240567599
Mean Squared Error 17: 45.76979845164277
Mean Squared Error 18: 49.66988487614011
Mean Squared Error 19: 40.33367682892142
Mean Squared Error 20: 43.41892239962581
Mean Squared Error 21: 41.401803322208735
Mean Squared Error 22: 47.9685251797673
Mean Squared Error 23: 41.59414601248831
Mean Squared Error 24: 44.7113098588962
Mean Squared Error 25: 